In [3]:
import logging
import os.path

import numpy as np
import pymc as pm
import arviz as az
import aesara.tensor as at
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy import stats
import pandas as pd
import warnings
import extract_correct_csv

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [4]:

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

scaler = StandardScaler()

valid_subject = extract_correct_csv.extract_only_valid_subject()
valid_k_list = list(range(1, 10))

k=5
i =2
num_trials_to_remove = 48

string_subject = extract_correct_csv.read_correct_subject_csv(i)
csv_ = 'data/LookAtMe_0' + string_subject + '.csv'
# csv_ = '/home/paolo/matteo/matteo/unimi/tesi_master/code/osfstorage-archive/behavior/LookAtMe_045.csv'
global_data = pd.read_csv(csv_, sep='\t')
y = np.array(list([int(d > 2) for d in global_data['rating']]))
e_labels = y[:, np.newaxis]  # rating > 2
e_labels = e_labels[num_trials_to_remove:]


TRIAL = 160

hr = pd.read_csv('data/features/hr/'+str(i)+'.csv')
hr = hr[num_trials_to_remove:]

eda = pd.read_csv('data/features/eda/' + str(i) + '.csv')
eda = eda[num_trials_to_remove:]

pupil = pd.read_csv('data/features/pupil/' + str(i) + '.csv')
pupil = pupil[num_trials_to_remove:]


In [5]:
TRIAL_DEF = TRIAL - num_trials_to_remove
TRIAL_DEF

112

In [6]:
#TRAIN_PERC = 0.75
TEST_PERC = 0.25            #1-TRAIN_PERC
N_train = int(len(pupil) * (1-TEST_PERC))
N_train

84

In [7]:
pupil_train = pupil[:N_train]
hr_train = hr[:N_train]
eda_train = eda[:N_train]
e_labels_train = e_labels[:N_train]

In [8]:
N_pupil = pupil_train.shape[0]
D_pupil = pupil_train.shape[1]

N_hr = hr_train.shape[0]
D_hr = hr_train.shape[1]

N_eda = eda_train.shape[0]
D_eda = eda_train.shape[1]

N_e = e_labels_train.shape[0]
D_e = e_labels_train.shape[1]

K = k

In [9]:
print(N_pupil, D_pupil)
print(N_hr, D_hr)
print(N_eda, D_eda)
print(N_e, D_e)

84 5
84 60
84 60
84 1


In [10]:
with pm.Model() as sPPCA:
    sPPCA.add_coord('physio_n', np.arange(N_hr), mutable=True)
    sPPCA.add_coord('physio_d', np.arange(D_hr), mutable=False)
    sPPCA.add_coord('e_label_d', np.arange(D_e), mutable=True)
    sPPCA.add_coord('K', np.arange(K), mutable=True)
    sPPCA.add_coord('pupil_d',np.arange(D_pupil),mutable=True)

    # dati osservabili
    hr_data = pm.MutableData("hr_data", hr_train.T, dims=['physio_d', 'physio_n'])
    pupil_data = pm.MutableData("pupil_data", pupil_train.T, dims=['pupil_d', 'physio_n'])
    eda_data = pm.MutableData("eda_data", eda_train.T, dims=['physio_d', 'physio_n'])

    #e_data = pm.MutableData("e_data", e_labels_train.T)

    # matrici pesi
    Whr = pm.Normal('Whr', mu=0, sigma=2.0 * 1, dims=['physio_d', 'K'])
    Wpupil = pm.Normal('Wpupil', mu=0, sigma=2.0 * 1, dims=['pupil_d','K'])

    Weda = pm.Normal('Weda', mu=0, sigma=2.0 * 1, dims= ['physio_d','K'])

    # weight matrix for pain expectation.
    # check mu,sigma,shape
    We = pm.Normal('W_e', mu=0, sigma=2.0 * 1, dims=['e_label_d', 'K'])

    # latent space
    c = pm.Normal('c', mu=0, sigma=1, dims=['physio_n', 'K'])

    # dati dell'hrv interpretati come una gaussiana
    mu_hr = pm.Normal('mu_hr', Whr.dot(c.T), 1, dims=['physio_d', 'physio_n'])  # hyperprior 1
    sigma_hr = pm.Exponential('sigma_hr', 1)  # hyperprior 2
    x_hr = pm.Normal('x_hr', mu=mu_hr, sigma=sigma_hr, observed=hr_data, dims=['physio_d', 'physio_n'])

    # dati della dilatazione pupille interpretati come una gaussiana
    mu_pupil = pm.Normal('mu_pupil', Wpupil.dot(c.T), 1,dims=['pupil_d','physio_n'])  # hyperprior 1
    sigma_pupil = pm.Exponential('sigma_pupil', 1)  # hyperprior 2
    x_pupil = pm.Normal('x_pupil', mu=mu_pupil, sigma=sigma_pupil, dims=['pupil_d','physio_n'], observed=pupil_data)

    # eda
    mu_eda = pm.Normal('mu_eda', Weda.dot(c.T), 1, dims=['physio_d', 'physio_n'])  # hyperprior 1
    sigma_eda = pm.Exponential('sigma_eda', 1)  # hyperprior 2
    x_eda = pm.Normal('x_eda', mu=mu_eda, sigma=sigma_eda,  dims=['physio_d', 'physio_n'], observed=eda_data)

    # pain expectation. ciò che dovremmo inferire dato c
    # due strade: binary o multiclass (1-4)
    # p = probability of success?
    x_e = pm.Bernoulli('x_e', p=pm.math.sigmoid(We.dot(c.T)), dims=['e_label_d', 'physio_n'], observed=e_labels_train.T)

    # x_hr = pm.Bernoulli('x_hr', p=pm.math.sigmoid(Whr.dot(c.T)), shape=[D_hr, N_hr], observed=hr_data)
    # x_eda = pm.Bernoulli('x_eda', p=pm.math.sigmoid(Weda.dot(c.T)), shape=[D_eda, N_eda], observed=eda_data)

In [11]:
gv = pm.model_to_graphviz(sPPCA)
gv.view('unpooled_model')

'unpooled_model.pdf'

In [12]:
with sPPCA:
    approx = pm.fit(100000, callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)])
    trace = approx.sample(500)

qt5ct: using qt5ct plugin
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/32/"
Invalid Context= "Apps" line

Finished [100%]: Average Loss = 1.9195e+10


In [33]:
name = 'unpooled/k'+str(k)+'_sub'+str(i)+'_'

In [34]:
trace.posterior.to_netcdf(name+'posterior.h5', engine='scipy')

In [27]:
#from xarray import open_dataset

#posterior = open_dataset('posterior.h5', engine='scipy')

In [14]:
pupil_test = pupil[N_train:].reset_index().drop(columns=['index'])
hr_test = hr[N_train:].reset_index().drop(columns=['index'])
eda_test = eda[N_train:].reset_index().drop(columns=['index'])
e_test = e_labels[N_train:]
len(hr_test)

28

In [15]:
with sPPCA:
    posterior_pred = pm.sample_posterior_predictive(
        trace,var_names=["x_e"], random_seed=123)

Sampling: [x_e]


In [16]:
posterior_pred.posterior_predictive['x_e'].shape

(1, 500, 1, 84)

In [17]:
# az.plot_trace(trace);
with sPPCA:
    # update values of predictors:
    sPPCA.set_data("hr_data", hr_test.T, coords={'physio_n': range(hr_test.shape[0])})
    sPPCA.set_data("pupil_data", pupil_test.T, coords={'physio_n': range(pupil_test.shape[0])})
    sPPCA.set_data("eda_data", eda_test.T, coords={'physio_n': range(eda_test.shape[0])})
    # use the updated values and predict outcomes and probabilities:

    posterior_predictive = pm.sample_posterior_predictive(
        trace,var_names=["x_e"], random_seed=123, predictions=True)

Sampling: [c, x_e]


In [18]:
e_pred = posterior_predictive.predictions['x_e']
e_pred_mode = np.squeeze(stats.mode(e_pred[0], keepdims=False)[0])[:, np.newaxis]
e_pred.shape

(1, 500, 1, 28)

In [19]:
e_test

array([[1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1]])

In [20]:
train_accuracy_exp = accuracy_score(e_test, e_pred_mode)
print('\n\tTest Accuracy Pain Expectation: ' + str(train_accuracy_exp))
print(' ')


	Test Accuracy Pain Expectation: 0.42857142857142855
 
